In [6]:
# Day 5 challenge for AOC2019
# https://adventofcode.com/2019/day/5

In [4]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Memory:
    def __init__(self, size):
        self.size = size  # must specify the memory size
        self.storage = [None] * size  # initialize with None
            
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate)
        if(mode == 0):
            i = self.storage[mem_location]
            for j in codelist:
                self.storage[i] = j
                i+=1
        elif(mode == 1):
            i = mem_location
            for j in codelist:
                self.storage[i] = j
                i+=1

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode (position or immediate)
    def mode_read(self, pos, mode):
        # mode: 0 - position, 1 - immediate
        if(mode == 0):
            return self.storage[self.storage[pos]]
        elif(mode == 1):
            return self.storage[pos]

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

In [3]:
# this function will execute a single sequence (operator + subsequent operands and output location)
def exec_seq(ctr, m):
    # ctr - where to start
    # m - the memory object to work from
    # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
    # zero-pad to 5 - most significant 0 is implied
    enhanced_opcode = m.read(ctr).zfill(5)
    # first 3 characters are the parameter modes
    pmodes = enhanced_opcode[:2]
    # last 2 characters are the opcode
    opcode = enhanced_opcode[3:]
    # if it is 99, we're done
    if(opcode == 99):
        return("STOP")
    # this is the "add" operator
    elif(opcode == 1):
        operand1 = mode_read(ctr+1, pmodes[2])
        operand2 = mode_read(ctr+2, pmodes[1])
        calcval = operand1 + operand2
    # this is the "multiply" operator
    elif(opcode == 2):
        operand1 = mode_read(ctr+1, pmodes[2])
        operand2 = mode_read(ctr+2, pmodes[1])
        calcval = operand1 * operand2
    # this is the 'store from input' operator
    elif(opcode == 3):
        # get the input
        val = getfrominput()
        # store it in address provided by next code
        m.fill(val, m.read(ctr+1))
        # return next prog counter
        return(ctr+2)
    # this is the 'send to output' operator
    elif(opcode == 4):
        # read the val from location specified from nexy code and send to output
        sendtooutput(m.read(ctr+1))
        return(ctr+2)
    # if here, then the opcode is not known
    else:
        return("BADOPCODE")
    # fill the output destination with value depending on mode
    m.mode_fill([calcval], ctr+3, pmodes[0])
    # and return the next postion in the memory
    return(ctr+4)

In [9]:
# get the programming sequence
seq = [1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,10,1,19,2,19,6,23,2,13,23,27,1,9,27,31,2,31,9,35,1,6,35,39,2,10,39,43,1,5,43,47,1,5,47,51,2,51,6,55,2,10,55,59,1,59,9,63,2,13,63,67,1,10,67,71,1,71,5,75,1,75,6,79,1,10,79,83,1,5,83,87,1,5,87,91,2,91,6,95,2,6,95,99,2,10,99,103,1,103,5,107,1,2,107,111,1,6,111,0,99,2,14,0,0]
# create a computer object with twice the memory of that sequence
mycomp = Memory(len(seq) * 2)

In [10]:
# will replace values at pos 1 (noun) and 2 (verb) with 0..99 until the desired answer is found at pos 0
# reload Intcode program between each run
answer = 0
desired_answer = 19690720 

In [11]:
for i in range(0,99):
    for j in range(0,99):
        # reload the Intcode program
        mycomp.fill(seq)
        # put in the current noun and verb guesses
        mycomp.fill([i], 1)
        mycomp.fill([j], 2)

# primary loop
# start at the beginning...
        next_code = 0
# and keep going as long as integers keep coming back as the next code location (badness will be a string)
        while(isinstance(next_code, int)):
            next_code = exec_seq(next_code, mycomp)

# get the answer
        answer = mycomp.read(0)
        # check to see if it is the desired answer. if so, save the noun and verb.
        if(answer == desired_answer):
            noun = i
            verb = j

In [12]:
print("noun: {}, verb: {}".format(noun, verb))

noun: 64, verb: 29


In [13]:
# do the little math at the end of the challenge
submit_answer = 100 * noun + verb
print("submit_answer: {}".format(submit_answer))

submit_answer: 6429
